# 📤 Entregar Trabajo

**Instrucciones:**
1. Completa todos los ejercicios en este notebook
2. Ejecuta todas las celdas para verificar que funcionen
3. **Descarga este notebook**: `File → Download` (o `Archivo → Descargar`)
4. Ejecuta la celda de abajo para cargar el archivo descargado
5. Ingresa tu nombre y sube el archivo

**NO requiere cuenta ni login** - solo nombre y archivo.

In [ ]:
from IPython.display import display, HTML

html_form = """
<div style="padding:20px; background:#f0f0f0; border-radius:8px; max-width:600px; margin:20px auto;">
    <h3 style="margin-top:0">Entregar Trabajo</h3>
    
    <div style="margin:15px 0;">
        <label style="display:block; margin-bottom:5px; font-weight:bold;">Tu nombre completo:</label>
        <input id="studentName" type="text" placeholder="Ej: Juan Perez" 
               style="width:100%; padding:10px; font-size:16px; border:2px solid #ddd; border-radius:4px;"/>
    </div>
    
    <div style="margin:15px 0;">
        <label style="display:block; margin-bottom:5px; font-weight:bold;">Selecciona tu notebook (.ipynb):</label>
        <input id="fileInput" type="file" accept=".ipynb" 
               style="width:100%; padding:10px; font-size:16px; border:2px solid #ddd; border-radius:4px; background:white;"/>
    </div>
    
    <button id="submitBtn" 
            style="width:100%; padding:12px; background:#4CAF50; color:white; border:none; border-radius:4px; font-size:16px; cursor:pointer; font-weight:bold; margin-top:10px;">
        Entregar
    </button>
    
    <div id="result" style="margin-top:15px;"></div>
</div>

<script>
const SERVER_URL = "https://chaos-sub.vercel.app/submit";

document.getElementById('submitBtn').addEventListener('click', async function() {
    const resultDiv = document.getElementById('result');
    const nameInput = document.getElementById('studentName');
    const fileInput = document.getElementById('fileInput');
    
    const name = nameInput.value.trim();
    if (!name) {
        resultDiv.innerHTML = '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">Por favor ingresa tu nombre</div>';
        return;
    }
    
    if (!fileInput.files || fileInput.files.length === 0) {
        resultDiv.innerHTML = '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">Por favor selecciona un archivo .ipynb</div>';
        return;
    }
    
    const file = fileInput.files[0];
    if (!file.name.endsWith('.ipynb')) {
        resultDiv.innerHTML = '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">El archivo debe ser un notebook (.ipynb)</div>';
        return;
    }
    
    resultDiv.innerHTML = '<div style="padding:10px;background:#e7f3ff;border-radius:4px;color:#0066cc">Leyendo archivo...</div>';
    
    try {
        const reader = new FileReader();
        reader.onload = async function(e) {
            try {
                const notebookContent = JSON.parse(e.target.result);
                
                resultDiv.innerHTML = '<div style="padding:10px;background:#e7f3ff;border-radius:4px;color:#0066cc">Enviando al servidor...</div>';
                
                const payload = {
                    student_name: name,
                    notebook: notebookContent
                };
                
                const response = await fetch(SERVER_URL, {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify(payload)
                });
                
                const result = await response.json();
                
                if (response.ok) {
                    resultDiv.innerHTML = 
                        '<div style="padding:15px;background:#d4edda;border-radius:4px;color:#155724;border-left:4px solid #28a745">' +
                        '<strong>Trabajo entregado exitosamente!</strong><br>' +
                        '<small>Archivo: ' + result.filename + '</small></div>';
                    
                    nameInput.value = '';
                    fileInput.value = '';
                } else {
                    resultDiv.innerHTML = 
                        '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">Error: ' + 
                        (result.error || 'Error desconocido') + '</div>';
                }
                
            } catch (error) {
                resultDiv.innerHTML = '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">Error procesando archivo: ' + error.message + '</div>';
            }
        };
        
        reader.onerror = function() {
            resultDiv.innerHTML = '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">Error leyendo archivo</div>';
        };
        
        reader.readAsText(file);
        
    } catch (error) {
        resultDiv.innerHTML = '<div style="padding:10px;background:#fee;border-radius:4px;color:#d00">Error: ' + error.message + '</div>';
    }
});
</script>
"""

display(HTML(html_form))
print("Formulario de entrega listo")
print("Descarga tu notebook y subelo usando el formulario")